Change the code a little!

In [79]:
op = ['and', 'or', 'not', 'forall', 'exists', 'implies', 'iff']

Step-0: Read input from txt file

In [80]:
# def parse_formula_with_brackets(formula):
#     precedence = {'∧': 1, '∨': 1, '⊃': 2, '↔': 2}
#     op_to_word = {'∧': 'and', '∨': 'or', '⊃': 'implies', '↔': 'iff'}
    
#     # Add spaces around operators and parentheses for easier tokenization
#     for op in list(precedence.keys()) + ['(', ')']:
#         formula = formula.replace(op, f' {op} ')
#     tokens = formula.split()
    
#     def parse_expression(tokens):
#         def next_token():
#             return tokens.pop(0) if tokens else None

#         def parse_primary_expr(token):
#             if token == '(':
#                 expr = parse()
#                 assert next_token() == ')', "Unmatched parenthesis"
#                 return expr
#             return token

#         def parse():
#             expr = parse_primary_expr(next_token())
#             while tokens and tokens[0] in precedence:
#                 op = next_token()
#                 right_expr = parse_primary_expr(next_token())
#                 expr = [op_to_word[op], expr, right_expr]
#             return expr

#         return parse()

#     parsed_expression = parse_expression(tokens)
#     return parsed_expression

# # Correct the approach for handling bracketed expressions
# formula = "( p ∨ q ) ⊃ ( r ∧ s ) ↔ t"
# parsed_formula = parse_formula_with_brackets(formula)
# parsed_formula

Converting input formula into list form

In [81]:
import re

def parse_logical_expression(formula):
    # Updated to include '↔' operator
    op_map = {'∀': 'forall', '∃': 'exists', '¬': 'not', '∧': 'and', '∨': 'or', '⊃': 'implies', '↔': 'iff'}
    precedence = {'¬': 5, '∧': 4, '∨': 3, '⊃': 2, '↔': 1, '∀': -1, '∃': -1}

    formula = re.sub(r'([∀∃¬∧∨⊃↔()])', r' \1 ', formula)
    tokens = [token for token in re.split(r'\s+', formula) if token]

    def next_token():
        if tokens:
            return tokens.pop(0)
        return None

    def parse_primary():
        token = next_token()
        if token in ['∀', '∃']:
            var_token = next_token()
            expr = parse_expression()
            return [op_map[token], var_token, expr]
        elif token == '¬':
            expr = parse_primary()
            return [op_map[token], expr]
        elif token.isalnum():
            left_expr = token
            if tokens and tokens[0] == '(':
                tokens.pop(0)  # Remove '('
                args = []
                while tokens and tokens[0] != ')':
                    args.append(parse_expression())
                    if tokens and tokens[0] == ',':
                        tokens.pop(0)
                tokens.pop(0)  # Remove ')'
                left_expr = [token] + args
            return left_expr
        elif token == '(':
            expr = parse_expression()
            if tokens[0] == ')':
                tokens.pop(0)  # Remove ')'
            return expr
        else:
            return None

    def parse_expression():
        expr = parse_primary()
        while tokens and tokens[0] in precedence:
            op = next_token()
            if precedence[op] == -1:  # Handle quantifiers specially
                continue
            right_expr = parse_primary()
            expr = [op_map[op], expr, right_expr]
        return expr

    parsed_expression = parse_expression()
    return parsed_expression

# Test the updated function with the corrected operator handling
formula = '∀x (p(x) ↔ q(x))'
parsed_formula = parse_logical_expression(formula)
print(parsed_formula)


['forall', 'x', ['iff', ['p', 'x'], ['q', 'x']]]


In [82]:
formula = '∀x p(x) ⊃ ∀y (q(y) ⊃ r(y))'
parsed_formula = parse_logical_expression(formula)
parsed_formula

['forall',
 'x',
 ['implies', ['p', 'x'], ['forall', 'y', ['implies', ['q', 'y'], ['r', 'y']]]]]

In [83]:
formula = 'p(a) ∧ q(b)'
parsed_formula = parse_logical_expression(formula)
parsed_formula

['and', ['p', 'a'], ['q', 'b']]

In [84]:
formula = '∀x p(x)'
parsed_formula = parse_logical_expression(formula)
parsed_formula

['forall', 'x', ['p', 'x']]

In [85]:
formula = '∃x ¬p(x)'
parsed_formula = parse_logical_expression(formula)
parsed_formula

['exists', 'x', ['not', ['p', 'x']]]

In [86]:
formula = '∀x (p(x) ⊃ q(x))'
parsed_formula = parse_logical_expression(formula)
parsed_formula

['forall', 'x', ['implies', ['p', 'x'], ['q', 'x']]]

In [87]:
formula = '∀x ∃y (p(x , y) ⊃ q(y))'
parsed_formula = parse_logical_expression(formula)
parsed_formula

['forall', 'x', ['exists', 'y', ['implies', ['p', 'x', 'y'], ['q', 'y']]]]

In [88]:
formula = '¬p(a) ∨ q(b)'
parsed_formula = parse_logical_expression(formula)
parsed_formula

['or', ['not', ['p', 'a']], ['q', 'b']]

In [89]:
formula = '∀x (p(x) ∧ (∃y (q(y) ∨ ¬r(x , y))))'
parsed_formula = parse_logical_expression(formula)
parsed_formula

['forall',
 'x',
 ['and',
  ['p', 'x'],
  ['exists', 'y', ['or', ['q', 'y'], ['not', ['r', 'x', 'y']]]]]]

In [90]:
formula = 'p(a) ⊃ q(b) ⊃ r(c)'
parsed_formula = parse_logical_expression(formula)
parsed_formula

['implies', ['implies', ['p', 'a'], ['q', 'b']], ['r', 'c']]

In [91]:
formula = '∀x (p(x) ↔ q(x))'
parsed_formula = parse_logical_expression(formula)
parsed_formula

['forall', 'x', ['iff', ['p', 'x'], ['q', 'x']]]

In [92]:
formula = '∀x (p(x) ∨ q(x)) ∧ ∃y (r(y) ∧ ¬s(y))'
parsed_formula = parse_logical_expression(formula)
parsed_formula

['forall',
 'x',
 ['and',
  ['or', ['p', 'x'], ['q', 'x']],
  ['exists', 'y', ['and', ['r', 'y'], ['not', ['s', 'y']]]]]]

In [130]:
formula = '∀x (p(x) ⊃ q(x))'
parsed_formula = parse_logical_expression(formula)
parsed_formula

['forall', 'x', ['implies', ['p', 'x'], ['q', 'x']]]

Step-1: Standardize the variables

In [93]:
def standardize_quantified_variables(fol, var_occurrences=None, current_renames=None):
    if var_occurrences is None:
        var_occurrences = {}
    if current_renames is None:
        current_renames = {}

    # Directly return non-list elements.
    if not isinstance(fol, list):
        # Replace the variable with its renamed version if it's been renamed.
        return current_renames.get(fol, fol)

    operator, *args = fol

    if operator in ['forall', 'exists']:
        # For quantifiers, rename the variable if it's already been used.
        quantifier, var, scope = fol
        if var in var_occurrences:
            # Increment and rename the variable.
            new_var_name = f"{var}_{var_occurrences[var]}"
            var_occurrences[var] += 1
        else:
            # Initialize the occurrence counter for new variables.
            var_occurrences[var] = 1
            new_var_name = var

        # Update the renaming for the current scope.
        updated_renames = {**current_renames, var: new_var_name}
        # Recursively standardize the scope with the updated renaming.
        standardized_scope = standardize_quantified_variables(scope, var_occurrences, updated_renames)
        return [quantifier, new_var_name, standardized_scope]
    
    # Recursively apply standardization to the arguments of logical operators.
    return [operator] + [standardize_quantified_variables(arg, var_occurrences, current_renames) for arg in args]

# Test the function
fol_formula = ['or', ['forall', 'x', ['P', 'x']], ['exists', 'x', ['Q', 'x']]]
# fol_formula = ['forall', 'x', ['exists', 'y', ['or', ['not', ['P', 'x', 'y']], ['exists', 'x', ['Q', 'x', 'z']]]]]
standardized_fol = standardize_quantified_variables(fol_formula)
print("Standardized FOL Formula:", standardized_fol)

Standardized FOL Formula: ['or', ['forall', 'x', ['P', 'x']], ['exists', 'x_1', ['Q', 'x_1']]]


Input - The logic is in the form of list.

TODO - Read the txt file and convert it into list.

Step-2: Eliminating the Implication

###### Function that takes an implication and returns its equivalent form

In [94]:
def eliminateImplication(logic):
    
    result = []
    result.append('or')
    result.append(['not', logic[1]])
    result.append(logic[2])
    
    return result

In [95]:
logic = ['implies', ['and', 'A', 'B'], 'C']
print(eliminateImplication(logic)) # ['or', ['not', ['and', 'A', 'B']], 'C']

['or', ['not', ['and', 'A', 'B']], 'C']


###### Function that takes an equivalence and returns its equivalent form

In [96]:
def eliminateIFF(logic):
    
    result = []
    result.append('and')
    result.append(eliminateImplication(['implies', logic[1], logic[2]]))
    result.append(eliminateImplication(['implies', logic[2], logic[1]]))
    
    return result

In [97]:
logic = ['iff', ['and', 'A', 'B'], 'C']
print(eliminateIFF(logic)) # ['and', ['or', ['not', ['and', 'A', 'B']], 'C'], ['or', ['not', 'C'], ['and', 'A', 'B']]]

['and', ['or', ['not', ['and', 'A', 'B']], 'C'], ['or', ['not', 'C'], ['and', 'A', 'B']]]


###### Function that takes an sentence in FOL and return its equivalent form free of implication

In [98]:
def parseImplications(logic):
    
    # If it is an iff statement, replace logic with the one we get by eliminating iffs
    if logic[0] == 'iff' and len(logic) == 3:
        logic = eliminateIFF(logic)
    #If it is an implies statement, replace logic with the one we get by eliminating implies
    if logic[0] == 'implies' and len(logic) == 3:
        logic = eliminateImplication(logic)
    
    
    #For all the attributes in the logic repeat the process recursively
    for i in range(1, len(logic)):
        # print(logic[i])
        if len(logic[i]) > 1:
            logic[i] = parseImplications(logic[i])
       
    #return the final logic statement devoid of all implications and iffs
    return logic

In [99]:
logic = ['or', 'D', ['implies', ['and', 'A', 'B'], 'C']]
# logic = ['implies', ['implies', 'A', 'B'], 'C']
print(parseImplications(logic)) # ['or', 'D', ['or', ['not', ['and', 'A', 'B']], 'C']]

['or', 'D', ['or', ['not', ['and', 'A', 'B']], 'C']]


Step-3: Converting the negation of the implication

∀x,y(P(x,y)) will be written in list form as:

['forall', 'x', ['forall', 'y', 'f(x,y)']] for functions.

['forall', 'x', ['forall', 'y', ['P', 'x', 'y']]] for predicates.

In [100]:
def propagateNOT(logic):
    
    result = []

    # Checking if the logic is a single element (Redundant bcs of the parseImplications function)
    # if type(logic[1]) == str:
    #     return logic
    #Checking if the inward logic is OR then append AND
    if logic[1][0] == 'or':
        result.append('and')
    #Chicking if inward logic is AND then append OR
    elif logic[1][0] == 'and':
        result.append('or')
    #Checking if inward logic is NOT then return the logic alone
    elif logic[1][0] == 'not':
        return logic[1][1]
    elif logic[1][0] == 'forall':
        # Create a negated existential quantifier with the same variable and expression
        return ['exists', logic[1][1], propagateNOT(['not', logic[1][2]])]
    # Check if the logic is an existential quantifier (∃) then convert to universal (∀) with NOT
    elif logic[1][0] == 'exists':
        # Create a negated universal quantifier with the same variable and expression
        return ['forall', logic[1][1], propagateNOT(['not', logic[1][2]])]
    else:
        return logic
        
    #For all arguments of the inner list
    for i in range(1, len(logic[1])):
        #check if the first argument is another list
        if len(logic[1][i]) != 1:
            #recursively call to propogate not further inwards
            result.append(propagateNOT(['not', logic[1][i]]))
        else:
            #else append the negation of the single element
            result.append(['not', logic[1][i]])
     
    return result

In [101]:
logic = ['not', ['P', 'x', 'y']]
print(propagateNOT(logic)) # ['not', ['P', 'x', 'y']]

['not', ['P', 'x', 'y']]


In [102]:
logic = ['not', ['and', 'A', 'B']]
print(propagateNOT(logic)) # ['or', ['not', 'A'], ['not', 'B']]

['or', ['not', 'A'], ['not', 'B']]


In [103]:
logic = ['not', ['forall', 'x', 'f(x,y)']]
print(propagateNOT(logic)) # ['exists', 'x', 'A']

['exists', 'x', ['not', 'f(x,y)']]


In [104]:
logic = ['not', ['forall', 'x', ['P', 'x', 'y']]]
print(propagateNOT(logic)) # ['exists', 'x', ['not', ['P', 'x', 'y']]]

['exists', 'x', ['not', ['P', 'x', 'y']]]


###### Function that takes an sentence in FOL and pushes the negation(if any) to the atomic level

In [105]:
def parseNOTs(logic):
    
    #If it is a NOT statement, replace logic with the one we get by propogating not inwards
    if logic[0] == 'not' and len(logic) == 2 and len(logic[1]) != 1:
        logic = propagateNOT(logic)
    
    #For all the attributes in the logic repeat the process recursively
    for i in range(1, len(logic)):
        if len(logic[i]) > 1:
            logic[i] = parseNOTs(logic[i])
    
    #return the final logic statement with NOT propogated inwards
    return logic

In [106]:
logic = ['or', ['not', ['or', ['not', 'A'], 'B']], 'C']
print(parseNOTs(logic)) # ['or', ['and', 'A', ['not', 'B']], 'C']

['or', ['and', 'A', ['not', 'B']], 'C']


Step-4: Eliminating the Existential Quantifier

Note: ∀x(∃y(P(x,y))) after skolemization is P(?x, sk-y(?x)).

In [107]:
def skolemize(fol, universal_vars=[], skolem_counter={'constants': 0, 'functions': 0}):
    """
    Skolemize a FOL formula represented as a list, maintaining list structure.
    
    Args:
    fol: The FOL formula as a list.
    universal_vars: Variables currently universally quantified in the scope.
    skolem_counter: A dictionary with counters for Skolem constants and functions.
    
    Returns:
    The skolemized formula in list form.
    """
    if fol[0] == 'forall':
        # Process the formula within the scope of the universal quantifier, adding the variable to the scope
        return ['forall', fol[1]] + [skolemize(fol[2], universal_vars + [fol[1]], skolem_counter)]
    elif fol[0] == 'exists':
        if len(universal_vars) == 0:
            # Replace with a Skolem constant if no universal variables are in scope
            skc = f'skc{skolem_counter["constants"]}'
            skolem_counter["constants"] += 1
            # Replace the existential variable with the Skolem constant in the predicate
            return replace_variable(fol[2], fol[1], skc)
        else:
            # Replace with a Skolem function if universal variables are in scope
            skf = f'skf{skolem_counter["functions"]}(' + ','.join(universal_vars) + ')'
            skolem_counter["functions"] += 1
            return replace_variable(fol[2], fol[1], skf)
    elif isinstance(fol, list):
        # Recursively process each element of the list if it's a complex expression
        return [skolemize(part, universal_vars, skolem_counter) for part in fol]
    else:
        # Return the element unchanged if it's not a quantifier expression
        return fol

def replace_variable(expr, var, replacement):
    """
    Recursively replace a variable with a Skolem constant/function in an expression.
    
    Args:
    expr: The expression (part of the formula) in which to replace the variable.
    var: The variable to replace.
    replacement: The Skolem constant or function to replace the variable with.
    
    Returns:
    The expression with the variable replaced.
    """
    if isinstance(expr, str):
        # If the expression is a string, replace the variable if it matches
        return expr.replace(var, replacement)
    elif isinstance(expr, list):
        # If the expression is a list, recursively replace in each element
        return [replace_variable(part, var, replacement) for part in expr]
    else:
        # Return the expression unchanged if it's neither a string nor a list
        return expr

# Example FOL formula
fol_formula = ['forall', 'x', ['exists', 'y', ['P', 'x', 'f(y)']]]
skolemized_formula = skolemize(fol_formula)
print(f"Skolemized Formula: {skolemized_formula}")

Skolemized Formula: ['forall', 'x', ['P', 'x', 'f(skf0(x))']]


In [108]:
fol_formula = ['forall', 'x', ['exists', 'y', ['P', 'x', 'y']]]
skolemized_formula = skolemize(fol_formula)
print(f"Skolemized Formula: {skolemized_formula}")

Skolemized Formula: ['forall', 'x', ['P', 'x', 'skf1(x)']]


Question: What to do with universal quantifier? Just 'x' or '?x'?

Step-5: Removing the Universal Quantifier

In [109]:
def remove_universal_quantifiers(fol):
    """
    Remove universal quantifiers ('forall') from a FOL formula.

    Args:
    fol: The FOL formula as a nested list.

    Returns:
    The modified FOL formula as a nested list without 'forall'.
    """
    if not fol or not isinstance(fol, list):
        return fol  # Return as is if fol is not a list or is empty
    
    if fol[0] == 'forall':
        # Skip 'forall' and directly return the processing result of its child without including 'forall' in the output
        return remove_universal_quantifiers(fol[2])
    else:
        # Recursively process each element in the list for nested structures
        return [remove_universal_quantifiers(element) for element in fol]

# Example FOL formula
fol_formula = ['forall', 'x', ['and', ['or', 'A', 'B'], ['forall', 'y', ['P', 'x', 'y']]]]
# Removing universal quantifiers
modified_formula = remove_universal_quantifiers(fol_formula)
print(modified_formula)

['and', ['or', 'A', 'B'], ['P', 'x', 'y']]


Step-6: Distributing union over intersection

In [110]:
def isDistributionCandidate(logic):
    
    if logic[0] == 'or':
        for i in range(1, len(logic)):
            if len(logic[i]) > 1:
                if logic[i][0] == 'and':
                    return True
    return False

In [111]:
def parseDistribution(logic):
      
    #Check if the logic is a candidate for distribution of OR over ANDs
    if isDistributionCandidate(logic):
        logic = distributeOR(logic)
        
    #For all the attributes in the logic repeat the process recursively
    for i in range(1, len(logic)):
        if len(logic[i]) > 1:
            logic[i] = parseDistribution(logic[i])
            
    #Check if the logic is a candidate for distribution of OR over ANDs 
    if isDistributionCandidate(logic):
        logic = distributeOR(logic)
    
    #return distributed logic
    return simplify(logic)

In [112]:
def simplify(logic):
    
    #Only simplify when the literals to an operator are more than 2
    if len(logic) > 3 and logic[0] in op:
        #need to simplify
        logic = getSimplified(logic[0], logic[1:len(logic)-1], logic[len(logic) - 1])
        
    #Repeat recursively for all sub logics
    for i in range(1, len(logic)):
        if len(logic[i]) > 1:
            logic[i] = simplify(logic[i])
        
    #Only simplify when the literals to an operator are more than 2
    if len(logic) > 3 and logic[0] in op:
        #need to simplify
        logic = getSimplified(logic[0], logic[1:len(logic)-1], logic[len(logic) - 1])
        
    return logic

In [113]:
def getSimplified(operator, literals, current):
    
    result = []
    result.append(operator)
    
    #If the literal length is 1 append the literal else recursively call for simplification
    if len(literals) == 1:
        result.append(literals[0])
    else:
        result.append(getSimplified(operator, literals[0:len(literals)-1], literals[len(literals)-1]))
        
    #append the current 
    result.append(current)
    
    #return simplified logic
    return result

In [114]:
def distributeOR(logic):
    
    result = []
    #AND will propogate outwards
    result.append('and')
    
    #Check if both the lists are ands
    if logic[1][0] == 'and' and logic[2][0] == 'and':
        #Distribute the literals 
        result.append(parseDistribution(['or', logic[1][1], logic[2][1]]))
        result.append(parseDistribution(['or', logic[1][1], logic[2][2]]))
        result.append(parseDistribution(['or', logic[1][2], logic[2][1]]))
        result.append(parseDistribution(['or', logic[1][2], logic[2][2]]))
        
    else:
        #Either one is and and
        if logic[1][0] == 'and':
            
            #Check if the second argument is a list
            if len(logic[2]) > 2:
                #check if its a candidate for distribution
                if isDistributionCandidate(logic[2]):
                    logic[2] = parseDistribution(logic[2])
                    
                    #Distribute the literals 
                    result.append(parseDistribution(['or', logic[1][1], logic[2][1]]))
                    result.append(parseDistribution(['or', logic[1][1], logic[2][2]]))
                    result.append(parseDistribution(['or', logic[1][2], logic[2][1]]))
                    result.append(parseDistribution(['or', logic[1][2], logic[2][2]]))
                
                else:
                    #Keep the second as it is
                    result.append(parseDistribution(['or', logic[1][1], logic[2]]))
                    result.append(parseDistribution(['or', logic[1][2], logic[2]]))
                    
            else:
                #Keep the second as it is
                result.append(parseDistribution(['or', logic[1][1], logic[2]]))
                result.append(parseDistribution(['or', logic[1][2], logic[2]]))
        else:
            
            #Check if the second argument is a list
            if len(logic[1]) > 2:
                #check if its a candidate for distribution
                if isDistributionCandidate(logic[1]):
                    logic[1] = parseDistribution(logic[1])
                    
                    #Distribute the literals 
                    result.append(parseDistribution(['or', logic[1][1], logic[2][1]]))
                    result.append(parseDistribution(['or', logic[1][1], logic[2][2]]))
                    result.append(parseDistribution(['or', logic[1][2], logic[2][1]]))
                    result.append(parseDistribution(['or', logic[1][2], logic[2][2]]))
                else:
                    #Keep the second as it is
                    result.append(parseDistribution(['or', logic[1], logic[2][1]]))
                    result.append(parseDistribution(['or', logic[1], logic[2][2]]))
            else:
                #Keep the second as it is
                result.append(parseDistribution(['or', logic[1], logic[2][1]]))
                result.append(parseDistribution(['or', logic[1], logic[2][2]]))
            
                
    return simplify(result)

In [115]:
logic = ['or', ['and', 'A', 'B'], ['and', 'C', 'D']]
print(parseDistribution(logic)) # ['and', ['or', 'A', 'C'], ['or', 'A', 'D'], ['or', 'B', 'C'], ['or', 'B', 'D']]

['and', ['and', ['and', ['or', 'A', 'C'], ['or', 'A', 'D']], ['or', 'B', 'C']], ['or', 'B', 'D']]


In [116]:
logic = ['or', ['and', 'A', 'B'], ['P', 'x', 'y', 'z']]
print(parseDistribution(logic)) # ['and', ['or', 'A', ['P', 'x', 'y', 'z']], ['or', 'B', ['P', 'x', 'y', 'z']]]

['and', ['or', 'A', ['P', 'x', 'y', 'z']], ['or', 'B', ['P', 'x', 'y', 'z']]]


Eliminating the conjunctions

In [117]:
def extract_clauses(cnf_formula):
    """
    Extract individual clauses from a CNF formula.
    
    Args:
    cnf_formula: The CNF formula represented as a nested list.
    
    Returns:
    A list of clauses, where each clause is represented as a list of literals.
    """
    # Base case: if the formula is not a list, it's a single literal clause.
    if not isinstance(cnf_formula, list):
        return [[cnf_formula]]  # Return a single literal in a clause
    
    operator, *operands = cnf_formula
    
    # If the top-level operator is 'and', recursively extract clauses from operands.
    if operator == 'and':
        clauses = []
        for operand in operands:
            clauses.extend(extract_clauses(operand))  # Combine clauses from all 'and' operands
        return clauses
    
    # If the top-level operator is 'or', the whole formula is a single clause.
    elif operator == 'or':
        # Recursively flatten the 'or' structure into a single clause
        return [flatten_or(operands)]
    
    # If the formula does not start with 'and' or 'or', treat it as a single literal clause.
    else:
        return [[cnf_formula]]

def flatten_or(operands):
    """
    Flatten a nested 'or' structure into a single list of literals.
    
    Args:
    operands: A list of operands in an 'or' expression.
    
    Returns:
    A flattened list of literals.
    """
    flattened_clause = []
    for operand in operands:
        if isinstance(operand, list) and operand[0] == 'or':
            # If the operand is an 'or', recursively flatten and extend the clause
            flattened_clause.extend(flatten_or(operand[1:]))
        else:
            # Otherwise, add the operand (literal or sub-clause) directly to the clause
            flattened_clause.append(operand)
    return flattened_clause

# Example usage
cnf_formula = ['and', ['or', 'P', 'Q'], ['or', 'R', 'S']]
# cnf_formula = ['and', ['and', 'P', 'Q'], 'R']
clauses = extract_clauses(cnf_formula)
print(f"Clauses: {clauses}")

Clauses: [['P', 'Q'], ['R', 'S']]


Removing Duplicates

In [118]:
def remove_duplicates_from_clauses(clauses):
    """
    Remove duplicate literals within each clause and then remove duplicate clauses.
    
    Args:
    clauses: A list of clauses, where each clause is a list of literals.
    
    Returns:
    A list of unique clauses with unique literals within each clause.
    """
    # Remove duplicate literals within each clause
    unique_clauses = [list(set(clause)) for clause in clauses]
    
    # Remove duplicate clauses
    # Convert each clause to a tuple for hashability, then use a set to remove duplicates, then convert back to list
    unique_clauses = list(map(list, set(map(tuple, unique_clauses))))
    
    return unique_clauses

# Example clauses, including duplicates within and across clauses
clauses = [['P', 'Q'], ['R', 'S'], ['P', 'Q', 'Q'], ['R', 'S'], ['T']]

# Remove duplicates
unique_clauses = remove_duplicates_from_clauses(clauses)

unique_clauses

[['R', 'S'], ['T'], ['Q', 'P']]

Combining all the steps

In [122]:
def parseLogic(logic):
    
    #For Empty logic
    if len(logic) == 0:
        return []
    #For logic with one Literal
    if len(logic) == 1:
        return logic
    
    result = parseImplications(logic)
    result = parseNOTs(result)
    result = skolemize(result)
    # result = simplify(result)
    result = remove_universal_quantifiers(result)
    result = parseDistribution(result)
    result = extract_clauses(result)
    # result = remove_duplicates_from_clauses(result)
    
    return result

In [123]:
logic = ['or', ['and', 'A', ['forall', 'x', ['implies', ['P', 'x'], 'B']]], ['exists', 'y', ['and', ['P', 'y'], 'C']]]
print(parseLogic(logic))

[['A', ['P', 'skc2']], ['A', 'C'], [['not', ['P', 'x']], 'B', ['P', 'skc2']], [['not', ['P', 'x']], 'B', 'C']]


Reading Input from File

In [131]:
# Define the file name
file_name = 'formula.txt'

# Initialize a variable to hold the formula
formula = ''

# Open the file and read the formula
try:
    with open(file_name, 'r') as file:
        formula = str(file.read().strip())  # Read the content and strip any leading/trailing whitespace
except FileNotFoundError:
    print(f"The file {file_name} was not found.")

# Print the formula
print(formula)


∀x (p(x) ⊃ q(x))


In [135]:
new = parse_logical_expression(formula)
print(new)
result = parseLogic(new)
print(result)

['forall', 'x', ['implies', ['p', 'x'], ['q', 'x']]]
[[['not', ['p', 'x']], ['q', 'x']]]


Converting List Form to Operator Form

In [137]:
def logical_expression(clauses):
    processed_clauses = []
    for clause in clauses:
        if len(clause) == 1:
            processed_clauses.append(clause[0])
        else:
            temp = ""
            for literal in clause:
                if isinstance(literal, list) and literal[0] == 'not':
                    if len(literal[1]) > 1:
                        literal[1] = literal[1][0] + "(" + ", ".join(literal[1][1:]) + ")" + " V "
                    temp += (f'¬{literal[1]}')
                elif isinstance(literal, list):
                    if len(literal) > 1:
                        literal = literal[0] + "(" + ", ".join(literal[1:]) + ")" + " V "
                    temp += literal
                else:
                    temp += literal + " ∨ "
            processed_clauses.append(temp[:-3])
    
    return processed_clauses
    

# Example usage
clauses = [['A', ['P', 'skc2']], ['A', 'C'], ['B', ['not', ['P', 'x']], ['P', 'skc2']], [['P', 'x'], 'B', 'C']]
logical_expression = logical_expression(clauses)
for clause in logical_expression:
    print(clause)

A ∨ P(skc2)
A ∨ C
B ∨ ¬P(x) V P(skc2)
P(x) V B ∨ C


TODO: Rename all the variables so that no two clauses have the same variable name.

Simplify

Further Steps:

How does the parser differentiate between the variables and the constants?

How to return the output?